<a href="https://colab.research.google.com/github/ikyath/CardioVascularDiseaseDetection/blob/master/Cardiovascular_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pwd


In [0]:
import pandas as pd

In [0]:
ls

In [0]:
cd /content/drive/My\ Drive/Data\ Science/Hackathon

In [0]:
train = pd.read_csv('cardio_train.csv',sep=';')

In [0]:
train.head()

In [0]:
train.shape

In [0]:
train.info()

In [0]:
train.cholesterol.unique()

In [0]:
train.gluc.unique()

In [0]:
train.smoke.unique()

In [0]:
train.alco.unique()

In [0]:
train.active.unique()

In [0]:
train.cardio.unique()